## Final Project Overview
### Opportunity Statement

The Covid pandemic resulted in a greatly renewed interest in nature & outdoor activities, due to safety issues and restrictions placed on typical indoor attractions in urban areas. 

The Google Trends data below shows the enormous spike in searches for "Outdoor Activities" in the US in 2020 compared to the preceeding 5 years.

![title](trendsplot.png)

While one can assume some return closer to pre-pandemic baselines once the pandemic is addressed, interest in outdoor-oriented vacations/remote work can reasonably be expected to remain at a higer level than they were pre-pandemic.  

This project will attempt to assist an investor attempting to take advantage of this renewed interest in outdoor experience, by identifying smaller rural cities and towns with an abundance of outdoor activity.

My datasets will leverage the [Foursquare Places API](https://developer.foursquare.com/docs/places-api/) to identify outdoor places and the [Zillow Home Values Forecast (City Admin Level)](https://www.zillow.com/research/data/) to identify home values trends in these areas.  

I will then cluster the data using K-Means Clustering to identify ideal markets for a potential investment into a vacation rental home.

### Data Source Examples 
#### Zillow Home Values Forecast Data
Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. It reflects the typical value for homes in the 35th to 65th percentile range



In [46]:
import pandas as pd
zillow = pd.read_csv('zillowfcast.csv', header=0)
zillow.tail()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1996-01-31,1996-02-29,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31
25926,395428,29449,Little Grass Valley,City,CA,CA,NaN,Plumas County,NaN,NaN,...,282017.0,281305.0,280540.0,280845.0,280661.0,279527.0,277232.0,275612.0,273830.0,270432.0
25927,52073,29449,Fort Montgomery,City,NY,NY,New York-Newark-Jersey City,Orange County,NaN,NaN,...,307073.0,309881.0,310864.0,312366.0,313472.0,314879.0,315854.0,319144.0,322102.0,325701.0
25928,205478,29449,Lawrence Creek,City,OK,OK,Tulsa,Creek County,32827.0,33216.0,...,84880.0,84798.0,84873.0,85700.0,86802.0,87953.0,88825.0,89320.0,89620.0,89812.0
25929,13817,29449,Sheridan,City,TX,TX,NaN,Colorado County,NaN,NaN,...,125701.0,125679.0,125894.0,126312.0,126634.0,126402.0,126086.0,125938.0,124792.0,124475.0
25930,22360,29449,Rockport,City,KY,KY,NaN,Ohio County,NaN,NaN,...,70099.0,70976.0,71533.0,71923.0,71966.0,72504.0,72235.0,72659.0,72345.0,72476.0


#### Foursquare Places API
The Foursquare Places API will be used to identify Outdoor 

In [47]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip3 install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip3 install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [48]:
CLIENT_ID = 'ZBR105HPM3DMPXP02XFNLGL2W53HNO5OTGXKGEY1WS24VBTL' # your Foursquare ID
CLIENT_SECRET = 'PAUR52BKF1AQG4JKB4JCCB3WAQETBEWCNCTWMZRQPUJMUPQX' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZBR105HPM3DMPXP02XFNLGL2W53HNO5OTGXKGEY1WS24VBTL
CLIENT_SECRET:PAUR52BKF1AQG4JKB4JCCB3WAQETBEWCNCTWMZRQPUJMUPQX


In [49]:
address = '703 Main St, Ouray, CO 81427'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.0242108 -107.6712321


In [63]:
search_query = 'Trail'
radius = 5000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd44a674fe037099c612345'},
 'response': {'venues': [{'id': '51e30b94498e535deccfdea5',
    'name': 'Mt. Sneffels Trail Head',
    'location': {'lat': 38.02182290933693,
     'lng': -107.6715202730119,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.02182290933693,
       'lng': -107.6715202730119}],
     'distance': 267,
     'cc': 'US',
     'state': 'Colorado',
     'country': 'United States',
     'formattedAddress': ['Colorado', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d159941735',
      'name': 'Trail',
      'pluralName': 'Trails',
      'shortName': 'Trail',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1607748199',
    'hasPerk': False},
   {'id': '55ef161d498e8edb11ca3aec',
    'name': 'Ouray Perimeter Trail',
    'location': {'lat': 38.02943903062262,
     'lng': -107.67201167350797,
   

In [68]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(35)

<ipython-input-68-42eea7c284c9>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.state,location.country,location.formattedAddress,location.city,location.postalCode,location.address
0,51e30b94498e535deccfdea5,Mt. Sneffels Trail Head,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",v-1607748199,False,38.021823,-107.671520,"[{'label': 'display', 'lat': 38.02182290933693...",267,US,Colorado,United States,"[Colorado, United States]",NaN,NaN,NaN
1,55ef161d498e8edb11ca3aec,Ouray Perimeter Trail,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",v-1607748199,False,38.029439,-107.672012,"[{'label': 'display', 'lat': 38.02943903062262...",586,US,CO,United States,"[Ouray, CO, United States]",Ouray,NaN,NaN
2,5ce9cee5d807ee002c9dba02,Baby Bathtubs Trail,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",v-1607748199,False,38.017479,-107.665111,"[{'label': 'display', 'lat': 38.017479, 'lng':...",921,US,CO,United States,"[Ouray, CO 81427, United States]",Ouray,81427,NaN
3,57b4acdb498eab139da11578,Box Canyon Native Plant Trail,"[{'id': '52e81612bcbc57f1066b7a13', 'name': 'N...",v-1607748199,False,38.016914,-107.676650,"[{'label': 'display', 'lat': 38.016914, 'lng':...",941,US,CO,United States,"[Ouray, CO 81432, United States]",Ouray,81432,NaN
4,55b64564498eede043a66795,hayden trail,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",v-1607748199,False,38.004586,-107.691534,"[{'label': 'display', 'lat': 38.004586, 'lng':...",2818,US,Colorado,United States,"[Colorado, United States]",NaN,NaN,NaN
5,55b3c1a4498e1bf8d1e02158,Weehawken Trailhead,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",v-1607748199,False,38.026147,-107.673995,"[{'label': 'display', 'lat': 38.02614694067044...",324,US,Colorado,United States,"[Colorado, United States]",NaN,NaN,NaN
6,4ed2b19477c8ea62fed77ba4,Bear Creek Trailhead,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",v-1607748199,False,38.020705,-107.673451,"[{'label': 'display', 'lat': 38.02070464719809...",436,US,CO,United States,"[Ouray, CO 81427, United States]",Ouray,81427,NaN
7,4f2ad05fe4b0ce029fd4d100,Dallas Trailhead,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",v-1607748199,False,38.047891,-107.686754,"[{'label': 'display', 'lat': 38.047891, 'lng':...",2966,US,CO,United States,"[County Rd 17, Ouray, CO 81427, United States]",Ouray,81427,County Rd 17
